In [8]:
!pip install XlsxWriter


In [19]:
import pandas as pd

# === Step 1: Load files ===

# Load audit data (starts from 3rd row)
audit_df = pd.read_excel("../data/3_Encrypted_Audit_Sample_Cohort_149_Detailed_PCS.xlsx", header=1)

# Detect columns
uhid_col = next((col for col in audit_df.columns if "uhid" in col.lower()), None)
ipno_col = next((col for col in audit_df.columns if "ipno" in col.lower()), None)
if not uhid_col or not ipno_col:
    raise KeyError("Missing UHID or IPNO column in audit data.")

audit_df.rename(columns={uhid_col: "UHID", ipno_col: "IPNO"}, inplace=True)

# Load ventilation data
vent_df = pd.read_csv("../data/Ventilation Data Batch 1.csv")
ipno_vent_col = next((col for col in vent_df.columns if "ipno" in col.lower()), None)
if not ipno_vent_col:
    raise KeyError("Missing IPNO column in ventilation data.")
vent_df.rename(columns={ipno_vent_col: "IPNO"}, inplace=True)

# Load standardized column template
standard_df = pd.read_excel("../data/Standardized Patient Data Column Headers Jun 30.xlsx", header=1)
standard_cols = [col for col in standard_df.columns if not pd.isna(col)]

# === Step 2: Merge audit and ventilation data on IPNO ===
merged_df = audit_df.merge(vent_df, on="IPNO", how="left")

# === Step 3: Identify extra ventilation columns (excluding IPNO and Total Ventilation Hours) ===
vent_extra_cols = [col for col in vent_df.columns if col not in ["IPNO", "Total Ventilation Hours"]]

# === Step 4: Get all unique UHIDs ===
uhids = audit_df["UHID"].dropna().unique()

# === Step 5: Write Excel with a sheet per UHID ===
output_path = "UHID_Patient_Sheets.xlsx"
with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:
    for uhid in uhids:
        df_u = merged_df[merged_df["UHID"] == uhid].copy()

        # Get standard columns that exist
        final_cols = [col for col in standard_cols if col in df_u.columns]

        # Determine which vent_extra_cols are present in this UHID's data
        existing_vent_cols = [col for col in vent_extra_cols if col in df_u.columns]

        # If any of those columns have non-null values, insert them before Total Ventilation Hours
        if existing_vent_cols and not df_u[existing_vent_cols].isnull().all(axis=None):
            insert_at = final_cols.index("Total Ventilation Hours") if "Total Ventilation Hours" in final_cols else len(final_cols)
            for col in reversed(existing_vent_cols):
                final_cols.insert(insert_at, col)


        # Save to Excel sheet (limit to 31 characters)
        df_u[final_cols].to_excel(writer, sheet_name=str(uhid)[:31], index=False)

print(f"✅ Done! File saved as: {output_path}")


✅ Done! File saved as: UHID_Patient_Sheets.xlsx


In [2]:
print(audit_df.columns.tolist())


['eda6e9fa-b9b5-4e8c-bbe7-69dd3777d622', '29d84ac3-dccf-4980-99b2-0e743999ed60', 1, 89, '2023-07-06 07:18:14', '2023-07-08 12:42:09', 2, 'NON-AICVD', 5, 0, '0.1', '1.1', 'Discharged to home or self care (Routine Discharge)', 'CERVICAL SPONDOLYSIS', ' <PER> came to ED with aforementioned complaints. On arrival to ED his saturation was 65% in RA, PR-104/min and   blood pressure-180/100mmHg.On Chest auscultation,  bilateral crackles(+) .Chest Xray showed features of acute Pulmonary edema. He was started on with NTG infusion, NIV support and IV diuretics. He was transferred to CCU for further evaluation and management. ECHO showed dilated LA, RWMA, Moderate LV systolic dysfunction, mild MR, mild TR. Pulmonology opinion was taken in view of  X-Ray changes and Co2 Retention in ABG and advised nebulization.\n\nNIV, NTG and Lasix infusions were weaned off gradually as patient started clinically improving. Patient became symptomatically better and was shifted out to room. He was treated with an

In [12]:
import pandas as pd

# === File Paths ===
uhid_file_path = "../code/UHID_Patient_Sheets.xlsx"
standard_file_path = "../data/Standardized Patient Data Column Headers Jun 30.xlsx"

# === Load the First Sheet from the UHID Excel File ===
uhid_excel = pd.ExcelFile(uhid_file_path)
uhid_df = pd.read_excel(uhid_excel, sheet_name=uhid_excel.sheet_names[0])
uhid_columns = uhid_df.columns.tolist()

# === Load Standard Columns from Template (header on 2nd row → header=1) ===
standard_df = pd.read_excel(standard_file_path, header=1)
standard_columns = [col for col in standard_df.columns if not pd.isna(col)]

# === Identify Columns Missing in UHID Data ===
missing_columns = [col for col in standard_columns if col not in uhid_columns]

# === Output Results ===
print("\n⚠️ Missing Columns in UHID Sheet (present in Standard Template):\n")
for col in missing_columns:
    print(f"  - {col}")



⚠️ Missing Columns in UHID Sheet (present in Standard Template):

  - LABEL
  - SPECIALITY
  - CASESPLIT
  - SURGERYDATE
  - PATIENT_TYPE


In [18]:
import pandas as pd
import os

# Load the multi-sheet Excel file
input_path = "UHID_Patient_Sheets.xlsx"
output_path = "UHID_Patient_Sheets_Filtered.xlsx"

# Load all sheets
all_sheets = pd.read_excel(input_path, sheet_name=None, engine="openpyxl")

# Get all ventilation-related column names
# We'll collect from all sheets to be sure
ventilation_keywords = ['ventilation', 'ventilator']
vent_cols_set = set()

for sheet, df in all_sheets.items():
    for col in df.columns:
        col_lower = col.lower()
        if any(key in col_lower for key in ventilation_keywords) and col != "Total Ventilation Hours":
            ventilation_col = col.strip()
            ventilation_col = ventilation_col.replace("\n", " ").strip()
            ventilation_col = ventilation_col.replace("  ", " ")
            ventilation_col = ventilation_col.strip()
            ventilation_col = ventilation_col.strip(".")
            ventilation_col = ventilation_col.strip("_")
            ventilation_col = ventilation_col.strip("-")
            vent_cols_set.add(ventilation_col)

vent_cols = list(vent_cols_set)

# Create output Excel
os.makedirs("code", exist_ok=True)
with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    for sheet_name, df in all_sheets.items():
        if "Total Ventilation Hours" not in df.columns or df["Total Ventilation Hours"].isnull().all():
            # Drop all vent columns if 'Total Ventilation Hours' is missing or all null
            df = df.drop(columns=[col for col in vent_cols if col in df.columns], errors='ignore')
            df = df.drop(columns=["Total Ventilation Hours"], errors='ignore')

        df.to_excel(writer, sheet_name=sheet_name[:31], index=False)

print(f"✅ Done! Filtered file saved to: {output_path}")


✅ Done! Filtered file saved to: UHID_Patient_Sheets_Filtered.xlsx


In [20]:
import pandas as pd

# === Step 1: Load files ===

# Load audit data (starts from 3rd row)
audit_df = pd.read_excel("../data/3_Encrypted_Audit_Sample_Cohort_149_Detailed_PCS.xlsx", header=1)

# Detect columns
uhid_col = next((col for col in audit_df.columns if "uhid" in col.lower()), None)
ipno_col = next((col for col in audit_df.columns if "ipno" in col.lower()), None)
if not uhid_col or not ipno_col:
    raise KeyError("Missing UHID or IPNO column in audit data.")

audit_df.rename(columns={uhid_col: "UHID", ipno_col: "IPNO"}, inplace=True)

# Load ventilation data
vent_df = pd.read_csv("../data/Ventilation Data Batch 1.csv")
ipno_vent_col = next((col for col in vent_df.columns if "ipno" in col.lower()), None)
if not ipno_vent_col:
    raise KeyError("Missing IPNO column in ventilation data.")
vent_df.rename(columns={ipno_vent_col: "IPNO"}, inplace=True)

# Get list of IPNOs with ventilation data
vent_ipnos = set(vent_df["IPNO"].dropna().astype(str))

# Load standardized column template
standard_df = pd.read_excel("../data/Standardized Patient Data Column Headers Jun 30.xlsx", header=1)
standard_cols = [col for col in standard_df.columns if not pd.isna(col)]

# === Step 2: Merge audit and ventilation data on IPNO ===
merged_df = audit_df.merge(vent_df, on="IPNO", how="left")

# === Step 3: Define ventilation-related columns ===
vent_all_cols = [
    "use_1_start", "use_1_end", "use_1_duration",
    "use_2_start", "use_2_end", "use_2_duration",
    "use_3_start", "use_3_end", "use_3_duration",
    "use_4_start", "use_4_end", "use_4_duration",
    "use_5_start", "use_5_end", "use_5_duration",
    "use_6_start", "use_6_end", "use_6_duration",
    "use_7_start", "use_7_end", "use_7_duration",
    "DMV_DOA", "total_uses", "total_duration_hours"
]

# Always delete these columns even if ventilated
always_delete = [
    "use_6_start", "use_6_end", "use_6_duration",
    "use_7_start", "use_7_end", "use_7_duration"
]

# === Step 4: Get all unique UHIDs ===
uhids = audit_df["UHID"].dropna().unique()

# === Step 5: Write Excel with a sheet per UHID ===
output_path = "UHID_Patient_Sheets_Filtered.xlsx"
with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:
    for uhid in uhids:
        df_u = merged_df[merged_df["UHID"] == uhid].copy()

        # Get IPNO for this UHID (assume one IPNO per UHID for now)
        ipno = str(df_u["IPNO"].iloc[0])

        # Start with all standard columns that are present in df_u
        final_cols = [col for col in standard_cols if col in df_u.columns]

        # Always delete use_6 and use_7 related columns
        final_cols = [col for col in final_cols if col not in always_delete]

        # If IPNO not in ventilation list, drop other ventilation cols too
        if ipno not in vent_ipnos:
            final_cols = [col for col in final_cols if col not in vent_all_cols]

        # Save to Excel sheet (limit to 31 characters)
        df_u[final_cols].to_excel(writer, sheet_name=str(uhid)[:31], index=False)

print(f"✅ Done! File saved as: {output_path}")


✅ Done! File saved as: UHID_Patient_Sheets_Filtered.xlsx


In [24]:
import pandas as pd

# === Step 1: Load additional data ===
discharges_df = pd.read_excel("../data/1_Raw_Encrypted_Data.xlsx", sheet_name="Discharges_Data")
surgery_df = pd.read_excel("../data/1_Raw_Encrypted_Data.xlsx", sheet_name="Surgery_Data")

# Ensure UHID column names are consistent
uhid_col_discharges = next((col for col in discharges_df.columns if "uhid" in col.lower()), None)
uhid_col_surgery = next((col for col in surgery_df.columns if "uhid" in col.lower()), None)

discharges_df.rename(columns={uhid_col_discharges: "UHID"}, inplace=True)
surgery_df.rename(columns={uhid_col_surgery: "UHID"}, inplace=True)

# Extract required columns only
discharges_df = discharges_df[["UHID", "SPECIALITY", "CASESPLIT"]].drop_duplicates()
surgery_df = surgery_df[["UHID", "SURGERYDATE"]].drop_duplicates()

# === Step 2: Load existing filtered Excel with all UHID sheets ===
input_file = "UHID_Patient_Sheets_Filtered.xlsx"
output_file = "UHID_Patient_Sheets_Final.xlsx"

with pd.ExcelFile(input_file) as reader, pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
    for sheet_name in reader.sheet_names:
        df = reader.parse(sheet_name)

        # Extract UHID from sheet name
        uhid = sheet_name

        # Merge SPECIALITY, CASESPLIT
        df = df.merge(discharges_df, on="UHID", how="left")

        # Merge SURGERY Date
        df = df.merge(surgery_df, on="UHID", how="left")

        # Save updated sheet
        df.to_excel(writer, sheet_name=sheet_name[:31], index=False)

print(f"✅ Updated file saved as: {output_file}")


✅ Updated file saved as: UHID_Patient_Sheets_Final.xlsx


In [30]:
import pandas as pd
from itertools import groupby
from operator import itemgetter

# === Step 1: Load standard multi-row column header file ===
header_df = pd.read_excel("../data/Standardized Patient Data Column Headers Jun 30.xlsx", header=[0, 1])
multiindex_cols = header_df.columns

# Map: detailed column name -> category
col_mapping = {detailed: category for category, detailed in multiindex_cols if not pd.isna(detailed)}

# Ordered list of standard columns (detailed only)
standard_order = [detailed for (_, detailed) in multiindex_cols if not pd.isna(detailed)]

# === Step 2: Load UHID_Patient_Sheets_Final and format ===
input_file = "UHID_Patient_Sheets_Final.xlsx"
output_file = "UHID_Patient_Sheets_Formatted.xlsx"

with pd.ExcelFile(input_file) as reader, pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
    for sheet_name in reader.sheet_names:
        df = reader.parse(sheet_name)

        # Split columns into standard and extra
        available_cols = [col for col in standard_order if col in df.columns]
        extra_cols = [col for col in df.columns if col not in standard_order]

        # Final column order: standard + extra
        final_cols = available_cols + extra_cols
        df = df[final_cols]

        # Build category and detailed headers
        detailed_headers = final_cols
        category_headers = [col_mapping.get(col, "") for col in detailed_headers]

        # Write data starting from row 2
        df.to_excel(writer, sheet_name=sheet_name[:31], index=False, startrow=2, header=False)

        # Write category and detailed header rows
        worksheet = writer.sheets[sheet_name[:31]]

        for col_idx, (cat, colname) in enumerate(zip(category_headers, detailed_headers)):
            worksheet.write(0, col_idx, cat)
            worksheet.write(1, col_idx, colname)

        # Merge same category cells (visual only)
        grouped = [(key, list(group)) for key, group in groupby(enumerate(category_headers), key=lambda x: x[1])]
        for category, group in grouped:
            indices = [i for i, _ in group]
            if category and len(indices) > 1:
                worksheet.merge_range(0, indices[0], 0, indices[-1], category)

print(f"✅ Final formatted Excel saved as: {output_file}")


✅ Final formatted Excel saved as: UHID_Patient_Sheets_Formatted.xlsx


In [29]:
import pandas as pd
import zipfile
import os

# === Config ===
input_file = "UHID_Patient_Sheets_Formatted.xlsx"
output_dir = "individual_patients"
zip_file = "All_Patient_Files.zip"

# === Step 1: Create output directory ===
os.makedirs(output_dir, exist_ok=True)

# === Step 2: Read and save each sheet with flattened headers ===
with pd.ExcelFile(input_file) as reader:
    for i, sheet_name in enumerate(reader.sheet_names, start=1):
        df = reader.parse(sheet_name, header=[0, 1])  # MultiIndex

        # Flatten MultiIndex columns
        df.columns = [' - '.join([str(level) for level in col if str(level) != 'nan']).strip() for col in df.columns]

        # Save as Patient1.xlsx, Patient2.xlsx, ...
        out_filename = os.path.join(output_dir, f"Patient{i}.xlsx")
        df.to_excel(out_filename, index=False)

print("✅ All individual Excel files saved with flattened headers.")

# === Step 3: Zip all Excel files ===
with zipfile.ZipFile(zip_file, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for filename in os.listdir(output_dir):
        filepath = os.path.join(output_dir, filename)
        zipf.write(filepath, arcname=filename)

print(f"✅ All files zipped into: {zip_file}")


✅ All individual Excel files saved with flattened headers.
✅ All files zipped into: All_Patient_Files.zip


In [31]:
import pandas as pd
import zipfile
import os

# === Config ===
input_file = "UHID_Patient_Sheets_Formatted.xlsx"
output_dir = "individual_patients"
zip_file = "All_Patient_Files.zip"

# === Step 1: Create output directory ===
os.makedirs(output_dir, exist_ok=True)

# === Step 2: Read and save each sheet EXACTLY as-is ===
with pd.ExcelFile(input_file) as reader:
    for i, sheet_name in enumerate(reader.sheet_names, start=1):
        df = reader.parse(sheet_name, header=[0, 1])  # ✅ preserves two-level header

        out_filename = os.path.join(output_dir, f"Patient{i}.xlsx")
        
        # ✅ Write MultiIndex headers with index=True to avoid Pandas bug
        df.to_excel(out_filename, index=True)  # index=True is required for MultiIndex header write

print("✅ All individual Excel files saved exactly as-is.")

# === Step 3: Zip the files ===
with zipfile.ZipFile(zip_file, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for filename in os.listdir(output_dir):
        if filename.endswith(".xlsx"):
            filepath = os.path.join(output_dir, filename)
            zipf.write(filepath, arcname=filename)

print(f"✅ Zipped to: {zip_file}")


✅ All individual Excel files saved exactly as-is.
✅ Zipped to: All_Patient_Files.zip
